In [26]:
#importing library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor


In [2]:
#showing the dataset
df=pd.read_csv('../input/engineering-graduate-salary-prediction/Engineering_graduate_salary.csv')
#loading the dataset
df


,ID,Gender,DOB,10percentage,10board,12graduation,12percentage,12board,CollegeID,CollegeTier,...,MechanicalEngg,ElectricalEngg,TelecomEngg,CivilEngg,conscientiousness,agreeableness,extraversion,nueroticism,openess_to_experience,Salary
0,604399,f,1990-10-22,87.80,cbse,2009,84.00,cbse,6920,1,...,-1,-1,-1,-1,-0.1590,0.3789,1.2396,0.14590,0.2889,445000
1,988334,m,1990-05-15,57.00,cbse,2010,64.50,cbse,6624,2,...,-1,-1,-1,-1,1.1336,0.0459,1.2396,0.52620,-0.2859,110000
2,301647,m,1989-08-21,77.33,"maharashtra state board,pune",2007,85.17,amravati divisional board,9084,2,...,-1,-1,260,-1,0.5100,-0.1232,1.5428,-0.29020,-0.2875,255000
3,582313,m,1991-05-04,84.30,cbse,2009,86.00,cbse,8195,1,...,-1,-1,-1,-1,-0.4463,0.2124,0.3174,0.27270,0.4805,420000
4,339001,f,1990-10-30,82.00,cbse,2008,75.00,cbse,4889,2,...,-1,-1,-1,-1,-1.4992,-0.7473,-1.0697,0.06223,0.1864,200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,103174,f,1989-04-17,75.00,0,2005,73.00,0,1263,2,...,-1,-1,-1,-1,-1.1901,0.9688,-1.0697,1.35490,0.0284,120000
2994,352811,f,1991-07-22,84.00,state board,2008,77.00,state board,9481,2,...,-1,-1,-1,-1,-0.1082,0.0328,-0.4891,-0.29020,0.5024,120000
2995,287070,m,1988-11-24,91.40,bsemp,2006,65.56,bsemp,547,2,...,-1,-1,-1,-1,-0.8810,0.1888,-0.3440,0.06230,0.6603,385000
2996,317336,m,1988-08-25,88.64,karnataka education board,2006,65.16,karnataka education board,1629,2,...,-1,-1,-1,-1,1.4374,1.2808,-0.4891,-1.46537,0.5419,530000


In [3]:
#getting information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     2998 non-null   int64  
 1   Gender                 2998 non-null   object 
 2   DOB                    2998 non-null   object 
 3   10percentage           2998 non-null   float64
 4   10board                2998 non-null   object 
 5   12graduation           2998 non-null   int64  
 6   12percentage           2998 non-null   float64
 7   12board                2998 non-null   object 
 8   CollegeID              2998 non-null   int64  
 9   CollegeTier            2998 non-null   int64  
 10  Degree                 2998 non-null   object 
 11  Specialization         2998 non-null   object 
 12  collegeGPA             2998 non-null   float64
 13  CollegeCityID          2998 non-null   int64  
 14  CollegeCityTier        2998 non-null   int64  
 15  Coll

# Preprocessing Inputs

In [4]:
def onehot_encode(df,column):
    #copying the original dataframe
    df=df.copy()
    #creating dummies dataframe and storing it into a variable
    dummies=pd.get_dummies(df[column],prefix=column)
    #concating the original dataframe to dummies
    df=pd.concat([df,dummies],axis=1)
    #dropping the original dataframe
    df=df.drop(column,axis=1)
    #returning the dataframe
    return df

In [35]:
def preprocess_inputs(df):
    #copying he dataset into a new variable
    df=df.copy()
    #droppng the id column
    df=df.drop('ID',axis=1)
    #binary encoding
    df['Gender']=df['Gender'].replace({'f':0,'m':1})
    #Date encoding
    df['DOB']=pd.to_datetime(df['DOB'])
    df['DOB_year']=df['DOB'].apply(lambda x:x.year)
    df['DOB_month']=df['DOB'].apply(lambda x:x.month)
    df['DOB_day']=df['DOB'].apply(lambda x:x.day)
    df=df.drop('DOB',axis=1)
    #one-hot ecoding
    #for loop
    for column in ['10board','12board','Degree','Specialization','CollegeState']:
        #calling the function
        df=onehot_encode(df,column)
    #replacing the -1 with NaN
    df=df.replace(-1,np.NaN)
    #missing_column
    missing_columns=[column for column in df.columns if df.isna().sum()[column]>0]
    for column in missing_columns:
        df[column]=df[column].fillna(df[column].mean())
    #Split df into x and y
    y=df['Salary']
    x=df.drop('Salary',axis=1)
    #train_test_split
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=1)
    #using standardscaler
    scaler=StandardScaler()
    #fitting the dataframe
    scaler.fit(x_train)
    x_train=pd.DataFrame(scaler.transform(x_train),columns=x_train.columns,index=x_train.index)
    x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns,index=x_test.index)
    
    
    
    return x_train,x_test,y_train,y_test

# Training the Model

In [40]:
lin_model=LinearRegression()
lin_model.fit(x_train,y_train)
lin_model.score(x_test,y_test)

-1.328071834154108e+27

In [41]:
xgb_model=XGBRegressor()
xgb_model.fit(x_train,y_train)
xgb_r2=xgb_model.score(x_test,y_test)
print('XGB R^2 Score {:5f}'.format(xgb_r2))

XGB R^2 Score 0.087638


In [6]:
[column for column in df.columns if df.isna().sum()[column]>0]

[]

In [7]:
df.isna().sum().sum()

0

In [36]:
x_train,x_test,y_train,y_test=preprocess_inputs(df)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2098, 599)
(900, 599)
(2098,)
(900,)


In [9]:
x.select_dtypes('object')

""
0
1
2
3
4
...
2993
2994
2995
2996


In [23]:
x_test

,Gender,10percentage,12graduation,12percentage,CollegeID,CollegeTier,collegeGPA,CollegeCityID,CollegeCityTier,GraduationYear,...,CollegeState_Orissa,CollegeState_Punjab,CollegeState_Rajasthan,CollegeState_Sikkim,CollegeState_Tamil Nadu,CollegeState_Telangana,CollegeState_Union Territory,CollegeState_Uttar Pradesh,CollegeState_Uttarakhand,CollegeState_West Bengal
749,0.57258,-2.857650,-0.664029,0.049641,0.960140,0.281457,-0.829184,0.960140,-0.649902,-1.219068,...,-0.212935,-0.230679,-0.204225,-0.03089,-0.310054,-0.294081,-0.048877,1.853515,-0.168616,-0.229531
2664,0.57258,-0.272108,1.181180,-0.255015,2.291505,0.281457,-0.363343,2.291505,-0.649902,1.085080,...,-0.212935,-0.230679,-0.204225,-0.03089,-0.310054,-0.294081,-0.048877,-0.539515,-0.168616,-0.229531
1392,-1.74648,-2.300764,-1.894168,-1.168982,-0.489610,0.281457,-0.101229,-0.489610,-0.649902,-0.451018,...,-0.212935,-0.230679,-0.204225,-0.03089,-0.310054,-0.294081,-0.048877,1.853515,-0.168616,-0.229531
1916,0.57258,0.911275,-0.664029,1.546038,0.844060,0.281457,-0.580735,0.844060,1.538693,-1.219068,...,-0.212935,-0.230679,-0.204225,-0.03089,-0.310054,3.400423,-0.048877,-0.539515,-0.168616,-0.229531
2995,0.57258,1.358773,-1.279099,-0.796227,-0.965667,0.281457,0.188214,-0.965667,-0.649902,-1.219068,...,-0.212935,-0.230679,-0.204225,-0.03089,-0.310054,-0.294081,-0.048877,-0.539515,-0.168616,-0.229531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,0.57258,-0.152775,0.566110,-0.828485,-0.357605,0.281457,-0.319864,-0.357605,-0.649902,0.317031,...,-0.212935,-0.230679,-0.204225,-0.03089,-0.310054,-0.294081,-0.048877,-0.539515,-0.168616,-0.229531
1812,0.57258,0.424000,-0.664029,-1.921661,1.430330,0.281457,-0.083838,1.430330,-0.649902,-0.451018,...,4.696269,-0.230679,-0.204225,-0.03089,-0.310054,-0.294081,-0.048877,-0.539515,-0.168616,-0.229531
2501,-1.74648,-1.664323,0.566110,-0.667196,0.662396,0.281457,0.909957,0.662396,-0.649902,1.085080,...,-0.212935,-0.230679,-0.204225,-0.03089,-0.310054,-0.294081,-0.048877,-0.539515,-0.168616,-0.229531
1043,-1.74648,1.120107,-0.048959,1.035292,0.802363,0.281457,1.406855,0.802363,-0.649902,1.085080,...,-0.212935,-0.230679,-0.204225,-0.03089,3.225240,-0.294081,-0.048877,-0.539515,-0.168616,-0.229531


In [10]:
#selecting only object column
{column:len(x[column].unique()) for column in x.select_dtypes('object').columns}

{}

In [11]:
df['Degree'].unique()

array(['B.Tech/B.E.', 'M.Tech./M.E.', 'MCA', 'M.Sc. (Tech.)'],
      dtype=object)

In [12]:
df['Specialization'].unique()

array(['instrumentation and control engineering',
       'computer science & engineering',
       'electronics & telecommunications', 'biotechnology',
       'mechanical engineering', 'information technology',
       'electronics and communication engineering',
       'computer engineering', 'computer application',
       'computer science and technology', 'electrical engineering',
       'automobile/automotive engineering',
       'electronics and electrical engineering',
       'information science engineering', 'chemical engineering',
       'instrumentation engineering', 'electronics & instrumentation eng',
       'ceramic engineering', 'metallurgical engineering',
       'aeronautical engineering', 'electronics engineering',
       'electronics and instrumentation engineering',
       'applied electronics and instrumentation', 'civil engineering',
       'computer and communication engineering',
       'industrial & production engineering', 'computer networking',
       'other', '

In [31]:
x_train.isna().sum()

Gender                          0
10percentage                    0
12graduation                    0
12percentage                    0
CollegeID                       0
                               ..
CollegeState_Telangana          0
CollegeState_Union Territory    0
CollegeState_Uttar Pradesh      0
CollegeState_Uttarakhand        0
CollegeState_West Bengal        0
Length: 599, dtype: int64

In [37]:
x_train.isna().sum().sum()

0

In [38]:
x_test.isna().sum().sum()

0